In [1]:
#| default_exp reading
%load_ext autoreload
%autoreload 2

import sys
from fastcore.xtras import Path


In [2]:
#| hide
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Leitura
> Este módulo concentra funções para a leitura dos diversos arquivos que compõe a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.
* `AERONAUTICA` - Consolidação de diversas bases de dados públicas da aeronáutica 

In [3]:
# | export
from pathlib import Path
import json
from pprint import pprint as pp

import pandas as pd
from dotenv import find_dotenv, load_dotenv


load_dotenv(find_dotenv())

True

## MOSAICO
> O mosaico atualmente é composto por 2 bases complementares originárias de um banco `MongoDB`: 

-  `Estações`
- `Plano Básico`

In [4]:
#|eval: false
#|code_fold: true
folder = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'saida'

In [5]:
#|eval: false
#|code_fold: true

def read_df(name: str)->pd.DataFrame:
    return pd.read_parquet(folder / f'{name}.parquet.gzip', dtype_backend='numpy_nullable').astype("category")

In [6]:
df = read_df('telecom_discarded')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501761 entries, 0 to 7501760
Data columns (total 22 columns):
 #   Column                Dtype   
---  ------                -----   
 0   Frequência            category
 1   Entidade              category
 2   Fistel                category
 3   Serviço               category
 4   Estação               category
 5   Latitude              category
 6   Longitude             category
 7   Código_Município      category
 8   Município             category
 9   UF                    category
 10  Classe                category
 11  Classe_Emissão        category
 12  Largura_Emissão(kHz)  category
 13  Validade_RF           category
 14  Status                category
 15  Fonte                 category
 16  Multiplicidade        category
 17  Log                   category
 18  _id                   category
 19  Num_Ato               category
 20  Tipo_Estação          category
 21  Tecnologia            category
dtypes: category(22)
me

In [8]:
df.sample()

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log,_id,Num_Ato,Tipo_Estação,Tecnologia
1136769,170.79375,SEPETIBA TECON S/A,50011784520,019,689544995,-22.930833,-43.836111,3302007,<NA>,RJ,...,11.0,2032-02-21,<NA>,<NA>,<NA>,"[{""Processamento"": ""Registro expandido em Larg...",5fb6f55229e244,670562007.0,movelml,<NA>


In [25]:
i = df.loc[:, ['Frequência', 'Entidade', 'Fistel', 'Estação', 'Classe_Emissão', 'Largura_Emissão(kHz)', 'Log']].sample()
i

,Frequência,Entidade,Fistel,Estação,Classe_Emissão,Largura_Emissão(kHz),Log
60788,149.77,V H F RADIO COMUNICACOES COMERCIAL LTDA,50411556584,1000492025,F1E,7.6,"[{""Processamento"": ""Registro expandido em Larg..."


In [26]:
pp(json.loads(i.Log.item()))

[{'Coluna': 'Designação_Emissão',
  'Original': '7K60F1EJN',
  'Processamento': 'Registro expandido em Largura_Emissão(kHz) e '
                   'Classe_Emissão'},
 {'Processamento': 'Registro agrupado num único registro no arquivo final. '
                   "Colunas Consideradas: ['Frequência', 'Fistel', "
                   "'Código_Município', 'Longitude', 'Latitude', 'Classe', "
                   "'Serviço', 'Classe_Emissão', 'Largura_Emissão(kHz)']"}]


In [9]:
from extracao.constants import AGG_LICENCIAMENTO

In [24]:
grouped = df.groupby(AGG_LICENCIAMENTO, dropna=True, sort=False, observed=True)


In [25]:
grouped_stations = grouped['Estação']

In [12]:
df_sub = df.dropna(subset=AGG_LICENCIAMENTO).reset_index(drop=True)

In [13]:
df_sub

,_id,Fistel,Serviço,Entidade,UF,Estação,Classe,Latitude,Longitude,Município,Código_Município,Num_Ato,Validade_RF,Frequência,Tipo_Estação,Tecnologia,Log,Largura_Emissão(kHz),Classe_Emissão
0,6337604c530fc,50402602889,011,Cricom do Brasil Ltda,PR,1002522193,ML,-25.408333333333335,-49.255277777777664,Curitiba,4106902,501362016,2035-11-11,459.25,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W
1,6337604c58a20,50402602889,011,Cricom do Brasil Ltda,PR,1002522193,ML,-25.408333333333335,-49.255277777777664,Curitiba,4106902,501362016,2035-11-11,469.125,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W
2,6337604e5c24a,50403060400,011,LOOPCOM COMERCIO E SERVICOS LTDA - ME,SP,1008389304,ML,-23.630013888888833,-46.52363888888883,Santo André,3547809,26672019,2036-03-29,383.125,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1E
3,6337604f66b99,50403738725,011,JEVIN COMERCIO E SERVIÇOS LTDA,RJ,1009950832,ML,-22.376113888888835,-41.79130833333333,Macaé,3302403,47952022,2026-11-21,383.85,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1D
4,6337604f6fd0a,50403738725,011,JEVIN COMERCIO E SERVIÇOS LTDA,RJ,1012366712,ML,-21.867583333333332,-41.0012805555555,São João da Barra,3305000,47952022,2026-11-21,459.5875,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6813009,f698d5f802581547,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,1875.0,base,GSM,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",200.0,G7W
6813010,f698d5f802581549,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,874.5,base,WCDMA,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",5000.0,G9W
6813011,f698d5f80258154b,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,77872023,2028-11-29,2130.0,base,WCDMA,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",5000.0,G9W
6813012,f698d5f80258154d,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,1875.0,base,LTE,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",10000.0,G7W


In [27]:
grouped_stations = grouped_stations.agg(lambda x: ', '.join(list(x)))

In [30]:
grouped_stations.values[0]

'1002522193, 691399310, 691399328, 691399336, 691399344, 691399352, 691399360, 691399379, 691399387, 691399395, 691399409, 691399417, 691399425, 691399433, 691399441, 691399450, 691399468, 691399476, 691399484, 691399492, 691399506, 691399514, 691399522, 691399530, 691399549, 691399557, 691399565, 691399573, 691399581, 691399590, 691399603, 691399611, 691399620, 691399638, 691399646, 691399654, 691399662, 691399670, 691399689, 691399697, 691399700, 691399719, 691399727, 691399735, 691399743, 691399751, 691399760, 691399778, 691399786, 691399794, 691399808, 691399816, 691399824, 691399832, 691399840, 691399859, 691399867, 691399875, 691399883, 691399891, 691399905, 691399913, 691399921, 691399930, 691399948, 691399956, 691399964, 691399972, 691399980, 691399999, 691400008, 691400016, 691400024, 691400032, 691400040, 691400059, 691400067, 691400075, 691400083, 691400091, 691400105, 691400113, 691400121, 691400130, 691400148, 691400156, 691400164, 691400172, 691400180, 691400199, 69140020

: 

In [21]:
df_sub

,_id,Fistel,Serviço,Entidade,UF,Estação,Classe,Latitude,Longitude,Município,Código_Município,Num_Ato,Validade_RF,Frequência,Tipo_Estação,Tecnologia,Log,Largura_Emissão(kHz),Classe_Emissão,#Estação
0,6337604c530fc,50402602889,011,Cricom do Brasil Ltda,PR,1002522193,ML,-25.408333333333335,-49.255277777777664,Curitiba,4106902,501362016,2035-11-11,459.25,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W,889153
1,6337604c58a20,50402602889,011,Cricom do Brasil Ltda,PR,1002522193,ML,-25.408333333333335,-49.255277777777664,Curitiba,4106902,501362016,2035-11-11,469.125,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W,889004
2,6337604e5c24a,50403060400,011,LOOPCOM COMERCIO E SERVICOS LTDA - ME,SP,1008389304,ML,-23.630013888888833,-46.52363888888883,Santo André,3547809,26672019,2036-03-29,383.125,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1E,889152
3,6337604f66b99,50403738725,011,JEVIN COMERCIO E SERVIÇOS LTDA,RJ,1009950832,ML,-22.376113888888835,-41.79130833333333,Macaé,3302403,47952022,2026-11-21,383.85,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1D,"888993, 889010, 5816461, 5816462, 5816463, 581..."
4,6337604f6fd0a,50403738725,011,JEVIN COMERCIO E SERVIÇOS LTDA,RJ,1012366712,ML,-21.867583333333332,-41.0012805555555,São João da Barra,3305000,47952022,2026-11-21,459.5875,movelml,<NA>,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",7.6,F1W,889155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6813009,f698d5f802581547,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,1875.0,base,GSM,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",200.0,G7W,NaN
6813010,f698d5f802581549,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,874.5,base,WCDMA,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",5000.0,G9W,NaN
6813011,f698d5f80258154b,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,77872023,2028-11-29,2130.0,base,WCDMA,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",5000.0,G9W,NaN
6813012,f698d5f80258154d,04030023541,175,ALGAR TELECOM S/A,MG,1015574944,FB,-18.871461,-48.294458,UBERLANDIA,3170206,22212023,2028-11-29,1875.0,base,LTE,"[{""Coluna"": ""Designação_Emissão"", ""Processamen...",10000.0,G7W,NaN


In [22]:
len(grouped_stations.groups.values())

289896